In [ ]:
%pip install vanna["chromadb,openai,postgres"]

In [1]:
from openai import OpenAI
from vanna.openai.openai_chat import OpenAI_Chat
from vanna.chromadb.chromadb_vector import ChromaDB_VectorStore

In [2]:
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

In [3]:
class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        # Pass the custom client to the OpenAI_Chat constructor
        OpenAI_Chat.__init__(self, client=client, config=config)
        ChromaDB_VectorStore.__init__(self, config=config)

vn = MyVanna(config={'model': 'local-model'})

In [4]:
vn.connect_to_sqlite('patients.db')

In [5]:
vn.train(ddl="CREATE TABLE patients_details (Patient_ID, Age, Gender, Nationality, Emirate, Diagnosis_Date, Cancer_Type, Cancer_Stage, Treatment_Type, Treatment_Start_Date, Hospital, Primary_Physician, Outcome, Death_Date, Cause_of_Death, Smoking_Status, Comorbidities, Ethnicity, Weight, Height)")

Adding ddl: CREATE TABLE patients_details (Patient_ID, Age, Gender, Nationality, Emirate, Diagnosis_Date, Cancer_Type, Cancer_Stage, Treatment_Type, Treatment_Start_Date, Hospital, Primary_Physician, Outcome, Death_Date, Cause_of_Death, Smoking_Status, Comorbidities, Ethnicity, Weight, Height)


'62e22abb-854a-580d-9ac9-f814093197ec-ddl'

In [6]:
training_data = vn.get_training_data()
training_data

,id,question,content,training_data_type
0,62e22abb-854a-580d-9ac9-f814093197ec-ddl,None,"CREATE TABLE patients_details (Patient_ID, Age...",ddl


In [7]:
vn.ask(question="how many patients are there in database?", visualize=False, allow_llm_to_see_data=True)

Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1


SQL Prompt: [{'role': 'system', 'content': "You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE patients_details (Patient_ID, Age, Gender, Nationality, Emirate, Diagnosis_Date, Cancer_Type, Cancer_Stage, Treatment_Type, Treatment_Start_Date, Hospital, Primary_Physician, Outcome, Death_Date, Cause_of_Death, Smoking_Status, Comorbidities, Ethnicity, Weight, Height)\n\n===Response Guidelines \n1. If the provided context is sufficient, please generate a valid SQL query without any explanations for the question. \n2. If the provided context is almost sufficient but requires knowledge of a specific string in a particular column, please generate an intermediate SQL query to find the distinct strings in that column. Prepend the query with a comment saying intermediate_sql \n3. If the provided context is insufficient, ple

('SELECT COUNT(DISTINCT Patient_ID) FROM patients_details;',
    COUNT(DISTINCT Patient_ID)
 0                       10000,
 None)

In [9]:
generated_sql = vn.generate_sql(question="how many patients are there in database?")
generated_sql

Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1


SQL Prompt: [{'role': 'system', 'content': "You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE patients_details (Patient_ID, Age, Gender, Nationality, Emirate, Diagnosis_Date, Cancer_Type, Cancer_Stage, Treatment_Type, Treatment_Start_Date, Hospital, Primary_Physician, Outcome, Death_Date, Cause_of_Death, Smoking_Status, Comorbidities, Ethnicity, Weight, Height)\n\n===Response Guidelines \n1. If the provided context is sufficient, please generate a valid SQL query without any explanations for the question. \n2. If the provided context is almost sufficient but requires knowledge of a specific string in a particular column, please generate an intermediate SQL query to find the distinct strings in that column. Prepend the query with a comment saying intermediate_sql \n3. If the provided context is insufficient, ple

'SELECT COUNT(DISTINCT Patient_ID) FROM patients_details;'

In [10]:
vn.is_sql_valid(sql=generated_sql)

True

In [11]:
vn.run_sql(sql=generated_sql)

,COUNT(DISTINCT Patient_ID)
0,10000


In [ ]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on


: 